# Abstract Collider Prompts

This notebook first explains the datastructure used in (Rehder and Waldmann, 2017)
and then demonstrates how to **construct the RW17 domain components** using helper functions 
from `dataset_creation`, and **convert them into a structured DataFrame**.

We will:
1. **Create a domain dictionary** using `create_domain_dict`
2. **Expand it into a DataFrame** using `expand_domain_to_dataframe`
3. **Add inference tasks** to extend the dataset
4. **Generate verbalized prompts** for human evaluation


This step will:

Load and examine rw_17_domain_components (the predefined dataset components).
Break down its structure to understand:
Domain dictionary (specification of causal variables).
Graph structure (causal relationships).
Inference tasks (reasoning scenarios).
Explain how these elements combine to generate prompts for LLMs.


In [1]:
import os
import sys
import pprint
import pandas as pd

# Ensure Python finds the `src` directory
sys.path.append(os.path.abspath("../../src"))

# Import everything defined in `__all__`
from causalalign.dataset_creation import (
    rw_17_domain_components,
    graph_structures,
    inference_tasks_rw17,
    generate_prompt_dataframe,
    expand_domain_to_dataframe,
    expand_df_by_task_queries,
    create_domain_dict,
    verbalize_domain_intro,
    verbalize_causal_mechanism,
    verbalize_inference_task,
    append_dfs,
)

print("Dataset creation module imported successfully!")


Dataset creation module imported successfully!


# 1. Understanding the RW17 Dataset Structure

Before generating new datasets, we need to understand the structure of **RW17 domain components**.

## 🔹 What Is RW17?
RW17 presented humans with causal inference tasks and asked for their likelihood judgements. Each causal inference task was presented on four subsequent screens.
In the following, I will describe how I translated the experimental materials used by RW17 into nested dictionaries, which serve as the backbone to algorithmically  generate  the materials in RW17 in *textual form* such that we can prompt and compare LLMs' causal judgements. The following notebook explains how to algoritmically re-create the the textual form used by RW17 and also, how to easily create new prompts.

RW17 used 3 different knowledge domains, in particular economy, sociology, and weather, in which the inference tasks were thematically embedded.
Each domain specifies:
- **Variables (`C1`, `C2`, `E`)**: The causal variables / graph nodes, e.g., C1: interest rates
- **Variable Sense depending on binary values 0 or 1 for (`C1`, `C2`, `E`)**: e.g. for C1=1: *high* interest rates
- ** Counterbalance-dependent Sense Assignments (`p/m`)**: How we represent conditions in counterbalanced ways (optional, but used in RW17). Essentially, this flips the senses of what it means for the variable to be on (1) or off (0).

The verbalization of the prompt depends on the domain and:
- **Causal Mechanisms**: How the variables influence each other (e.g., specified by collider, graph or chain graph)
- **Inference Tasks**: The reasoning problems we ask an LLM to solve specified by 

By combining these components, we **generate structured natural language prompts** that can be used for causal reasoning tasks in an LLMs.


##  Understanding the RW17 Domain Dictionary

Each domain in `rw_17_domain_components` contains:
1. **Domain Name & Introduction**:  
   - Explains the overall knowledge structure / cover story the inference task is embedded in.
   
2. **Causal Variables (`C1`, `C2`, `E`)**:  
   - `C1` and `C2` are **causes**, and `E` is the **effect**.
   - Each variable has:
     - A **name** and **detailed description**.
     - `p_value` and `m_value`: Counterbalanced values.
     - **Explanation mappings**: How specific conditions lead to outcomes. (optional, plus, they are graph dependent!)

3. **Example: Economy Domain**
   - `C1`: **Interest Rates** (low vs. high)
   - `C2`: **Trade Deficit** (small vs. large)
   - `E`: **Retirement Savings** (high vs. low)


#### Make sure you understand the building blocks of the dictionary. 

in ``src/causalalign/dataset_creation/constants.py``, there are dictionaries that define the domain building blocks, causal mechanism for 3 different graph topologies (collider, fork, and chain), and inference tasks. Before re-creating the prompts used in RW17, let's first load them and get a feeling for the prompt structure:




##  How Graph Structures Define Causal Mechanisms

A **graph structure** specifies how causal variables (`C1`, `C2`, `E`) relate to each other.

### Example Graph Structures:
1. **Collider** (`C1 → E ← C2`)
   - `C1` and `C2` both cause `E`.

2. **Fork** (`C1 ← E → C2`)
   - `E` causes both `C1` and `C2`.

3. **Chain** (`C1 → C2 → E`)
   - `C1` causes `C2`, which then affects `E`.

Let's look at the graph structures dictionary that is already pre-defined in ``src/causalalign/dataset_creation/constants.py``


In [2]:
# Pretty-print available graph structures
pprint.pprint(graph_structures)


{'chain': {'causal_template': '{c1_sense} {c1_name} causes {c2_sense} '
                              '{c2_name}. And {c2_sense} {c2_name} causes '
                              '{e_sense} {e_name}.',
           'description': 'C1→C2→E'},
 'collider': {'causal_template': '{c1_sense} {c1_name} causes {e_sense} '
                                 '{e_name}. Also, {c2_sense} {c2_name} causes '
                                 '{e_sense} {e_name}.',
              'description': 'C1→E←C2'},
 'fork': {'causal_template': '{c1_sense} {c1_name} causes {e_sense} {e_name}. '
                             'Also, {c1_sense} {c1_name} causes {c2_sense} '
                             '{c2_name}.',
          'description': 'E←C1→C2'}}


##  How Inference Tasks Define the Final Prompt

Inference tasks specify **what the LLM needs to predict** given certain observations.

For example:
- `"a": {"query_node": "Ci", "observation": "Cj=1", "query": "Ci=?"}`
  - **Ask:** Given that `Cj=1`, what is the likely value of `Ci`?
  - This corresponds to **a causal reasoning question**.

Inference tasks work together with **domain dictionaries** and **graph structures** to create **verbalized prompts**.

### 🔗 How It All Connects:
1. **Domain Dictionary** → Specifies **variables and values**.
2. **Graph Structure** → Defines **causal relationships**.
3. **Inference Tasks** → Frame **the reasoning problem**.
4. **Prompt Verbalization** → Converts this into **natural language for LLMs**.



In [3]:
pprint.pprint(inference_tasks_rw17)

{'a': {'observation': 'E=1, Cj=1',
       'query': 'p(Ci=1|E=1, Cj=1)',
       'query_node': 'Ci=1'},
 'b': {'observation': 'E=1', 'query': 'p(Ci=1|E=1)', 'query_node': 'Ci=1'},
 'c': {'observation': 'E=1, Cj=0',
       'query': 'p(Ci=1|E=1, Cj=0)',
       'query_node': 'Ci=1'},
 'd': {'observation': 'Cj=1', 'query': 'p(Ci=1|Cj=1)', 'query_node': 'Ci=1'},
 'e': {'observation': 'Cj=0', 'query': 'p(Ci=1|Cj=0)', 'query_node': 'Ci=1'},
 'f': {'observation': 'E=0, Cj=1',
       'query': 'p(Ci=1|E=0, Cj=1)',
       'query_node': 'Ci=1'},
 'g': {'observation': 'E=0', 'query': 'p(Ci=1|E=0)', 'query_node': 'Ci=1'},
 'h': {'observation': 'E=0, Cj=0',
       'query': 'p(Ci=1|E=0, Cj=0)',
       'query_node': 'Ci=1'},
 'i': {'observation': 'Ci=0, Cj=0',
       'query': 'p(E=1|Ci=0, Cj=0)',
       'query_node': 'E=1'},
 'j': {'observation': 'Ci=0, Cj=1',
       'query': 'p(E=1|Ci=0, Cj=1)',
       'query_node': 'E=1'},
 'k': {'observation': 'Ci=1, Cj=1',
       'query': 'p(E=1|Ci=1, Cj=1)',
       

In [4]:
scocio_dict_const = rw_17_domain_components["sociology"]
scocio_dict_const

{'domain_name': 'sociology',
 'variables': {'C1': {'C1_name': 'urbanization',
   'C1_detailed': 'Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments.',
   'p_value': {'1': 'high', '0': 'normal'},
   'm_value': {'1': 'low', '0': 'normal'},
   'explanations': {'p_p': 'Big cities provide many opportunities for financial and social improvement.',
    'p_m': 'In big cities many people are competing for the same high-status jobs and occupations.',
    'm_p': 'People in rural areas are rarely career oriented, and so take time off from working and switch frequently between different "temp" jobs.',
    'm_m': 'The low density of people prevents the dynamic economic expansion needed for people to get ahead.'}},
  'C2': {'C2_name': 'interest in religion',
   'C2_detailed': 'Interest in religion is the degree to which the members of a society show a curiosity in religion issues or participate in organized religions.',
   

In [5]:
socio_df = expand_domain_to_dataframe(scocio_dict_const)
socio_df

,domain,C1,C1_values,C1_cntbl,C1_sense,C1_detailed,C2,C2_values,C2_cntbl,C2_sense,C2_detailed,E,E_values,E_cntbl,E_sense,E_detailed,cntbl_cond
0,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,p,low,Interest in religion is the degree to which th...,socio-economic mobility,1,p,high,Socioeconomic mobility is the degree to which ...,ppp
1,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,p,low,Interest in religion is the degree to which th...,socio-economic mobility,1,m,low,Socioeconomic mobility is the degree to which ...,ppm
2,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,Interest in religion is the degree to which th...,socio-economic mobility,1,p,high,Socioeconomic mobility is the degree to which ...,pmp
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,Interest in religion is the degree to which th...,socio-economic mobility,1,m,low,Socioeconomic mobility is the degree to which ...,pmm
4,sociology,urbanization,1,m,low,Urbanization is the degree to which the member...,interest in religion,1,p,low,Interest in religion is the degree to which th...,socio-economic mobility,1,p,high,Socioeconomic mobility is the degree to which ...,mpp
5,sociology,urbanization,1,m,low,Urbanization is the degree to which the member...,interest in religion,1,p,low,Interest in religion is the degree to which th...,socio-economic mobility,1,m,low,Socioeconomic mobility is the degree to which ...,mpm
6,sociology,urbanization,1,m,low,Urbanization is the degree to which the member...,interest in religion,1,m,high,Interest in religion is the degree to which th...,socio-economic mobility,1,p,high,Socioeconomic mobility is the degree to which ...,mmp
7,sociology,urbanization,1,m,low,Urbanization is the degree to which the member...,interest in religion,1,m,high,Interest in religion is the degree to which th...,socio-economic mobility,1,m,low,Socioeconomic mobility is the degree to which ...,mmm


In [6]:
verbalize_causal_mechanism(scocio_dict_const, socio_df, "collider", graph_structures)

'0    high\n1    high\n2    high\n3    high\n4     low\n5     low\n6     low\n7     low\nName: C1_sense, dtype: object 0    urbanization\n1    urbanization\n2    urbanization\n3    urbanization\n4    urbanization\n5    urbanization\n6    urbanization\n7    urbanization\nName: C1, dtype: object causes 0    high\n1     low\n2    high\n3     low\n4    high\n5     low\n6    high\n7     low\nName: E_sense, dtype: object 0    socio-economic mobility\n1    socio-economic mobility\n2    socio-economic mobility\n3    socio-economic mobility\n4    socio-economic mobility\n5    socio-economic mobility\n6    socio-economic mobility\n7    socio-economic mobility\nName: E, dtype: object. Also, 0     low\n1     low\n2    high\n3    high\n4     low\n5     low\n6    high\n7    high\nName: C2_sense, dtype: object 0    interest in religion\n1    interest in religion\n2    interest in religion\n3    interest in religion\n4    interest in religion\n5    interest in religion\n6    interest in religion\n7   

In [7]:
const_df_test_socio = generate_prompt_dataframe(
    domain_dict=scocio_dict_const,
    inference_tasks=inference_tasks_rw17,
    graph_type="collider",
    graph_structures=graph_structures,
    counterbalance_enabled=True,
)


In [8]:
# subset wher column cntbl_cond is equal to mmp, mpm, pmm
# display entire row
# pd.set_option("display.max_colwidth", None)
# pd.set_option("display.max_rows", None)
const_df_test_socio = const_df_test_socio[
    const_df_test_socio["cntbl_cond"].isin(["mmp", "mpm", "pmm"])
]
const_df_test_socio

,domain,C1,C1_values,C1_cntbl,C1_sense,C1_detailed,C2,C2_values,C2_cntbl,C2_sense,...,E_sense,E_detailed,cntbl_cond,task,query_node,observation,query,graph,prompt,prompt_category
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,a,C1=1,"E=1, C2=1","p(C1=1|E=1, C2=1)",collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,a,C2=1,"E=1, C1=1","p(C2=1|E=1, C1=1)",collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,b,C1=1,E=1,p(C1=1|E=1),collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,b,C2=1,E=1,p(C2=1|E=1),collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,c,C1=1,"E=1, C2=0","p(C1=1|E=1, C2=0)",collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,c,C2=1,"E=1, C1=0","p(C2=1|E=1, C1=0)",collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,d,C1=1,C2=1,p(C1=1|C2=1),collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,d,C2=1,C1=1,p(C2=1|C1=1),collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,e,C1=1,C2=0,p(C1=1|C2=0),collider,Sociologists seek to describe and predict the ...,single_numeric_response
3,sociology,urbanization,1,p,high,Urbanization is the degree to which the member...,interest in religion,1,m,high,...,low,Socioeconomic mobility is the degree to which ...,pmm,e,C2=1,C1=0,p(C2=1|C1=0),collider,Sociologists seek to describe and predict the ...,single_numeric_response


Now, let's explore how these components are combined by re-creating the prompts used in RW17 starting with re-creating the dictionaries that are stored in ``constants.py``!


# Step 1: Create Domain Dictionray:


In [9]:
# # Create individual domains
# economy_domain_dict = create_domain_dict(
#     domain="economy",
#     introduction="Economists seek to describe and predict the regular patterns of economic fluctuation. To do this, they study some important variables or attributes of economies. They also study how these attributes are responsible for producing or causing one another.",
#     C1_name="interest rates",
#     C1_detailed="Interest rates are the rates banks charge to loan money.",
#     C1_values={"1": "low", "0": "high"},
#     C2_name="trade deficits",
#     C2_detailed="A country's trade deficit is the difference between the value of the goods that a country imports and the value of the goods that a country exports.",
#     C2_values={"1": "small", "0": "large"},
#     E_name="retirement savings",
#     E_detailed="Retirement savings is the money people save for their retirement.",
#     E_values={"1": "high", "0": "low"},
#     counterbalance_enabled=True,
# )

# sociology_domain_dict = create_domain_dict(
#     domain="sociology",
#     introduction="Sociologists seek to describe and predict the regular patterns of societal interactions. To do this, they study some important variables or attributes of societies. They also study how these attributes are responsible for producing or causing one another.",
#     C1_name="urbanization",
#     C1_detailed="Urbanization is the degree to which the members of a society live in urban environments (i.e., cities) versus rural environments.",
#     C1_values={"1": "high", "0": "low"},
#     C2_name="interest in religion",
#     C2_detailed="Interest in religion is the degree to which the members of a society show a curiosity in religion issues or participate in organized religions.",
#     C2_values={"1": "low", "0": "high"},
#     E_name="socio-economic mobility",
#     E_detailed="Socioeconomic mobility is the degree to which the members of a society are able to improve their social and economic status.",
#     E_values={"1": "high", "0": "low"},
#     counterbalance_enabled=True,
# )

# weather_domain_dict = create_domain_dict(
#     domain="weather",
#     introduction="Meteorologists seek to describe and predict the regular patterns that govern weather systems. To do this, they study some important variables or attributes of weather systems. They also study how these attributes are responsible for producing or causing one another.",
#     C1_name="ozone levels",
#     C1_detailed="Ozone is a gaseous allotrope of oxygen (O3) and is formed by exposure to UV radiation.",
#     C1_values={"1": "high", "0": "low"},
#     C2_name="air pressure",
#     C2_detailed="Air pressure is force exerted due to concentrations of air molecules.",
#     C2_values={"1": "low", "0": "high"},
#     E_name="humidity",
#     E_detailed="Humidity is the degree to which the atmosphere contains water molecules.",
#     E_values={"1": "high", "0": "low"},
#     counterbalance_enabled=True,
# )


eldonia_domain_dict = create_domain_dict(
    domain="Eldonia",
    introduction="The mystical scholars of Eldonia seek to understand the hidden forces that shape the arcane realm. They study key magical attributes and their influence on one another, unveiling the interconnected web of mystical energies.",
    C1_name="glyph resonance",
    C1_detailed="Glyph resonance determines the strength with which ancient runes interact with magical conduits.",
    C1_values={"1": "harmonized", "0": "disrupted"},
    C2_name="ether tides",
    C2_detailed="Ether tides are the fluctuating currents of raw magic that flow through the arcane realm, shifting unpredictably.",
    C2_values={"1": "stable", "0": "chaotic"},
    E_name="mana crystallization",
    E_detailed="Mana crystallization refers to the process by which magical energy condenses into solid form, enhancing spellcraft.",
    E_values={"1": "abundant", "0": "scarce"},
    counterbalance_enabled=True,
)
random_domain_dict = create_domain_dict(
    domain="glooblarp",
    introduction="Scholars of the ancient art of Glooblarp seek to understand the deep and mysterious connections between fundamental essences. By studying key variables, they attempt to decipher the cryptic patterns of the universe.",
    C1_name="flimbor flux",
    C1_detailed="Flimbor flux determines the wavering intensity of the glooblarp field in relation to quantum zorp.",
    C1_values={"1": "sprockled", "0": "unflarned"},
    C2_name="wibsnag oscillation",
    C2_detailed="Wibsnag oscillation measures the rate at which floobnoot particles shift between metaphysical states.",
    C2_values={"1": "snarfulant", "0": "drizzled"},
    E_name="blorb containment",
    E_detailed="Blorb containment is the ability to maintain a stable pocket of pure blorb energy without unexpected fizzle events.",
    E_values={"1": "glonkified", "0": "plimbled"},
    counterbalance_enabled=True,
)

abstract_domain_dict = create_domain_dict(
    domain="xqt",
    introduction="",
    C1_name="fwp",
    C1_detailed="",
    C1_values={"1": "high", "0": "low"},
    C2_name="blg",
    C2_detailed="",
    C2_values={"1": "small", "0": "large"},
    E_name="drk",
    E_detailed="",
    E_values={"1": "large", "0": "small"},
    counterbalance_enabled=True,
)


very_abstract_domain_dict = create_domain_dict(
    domain="zorpentix",
    introduction="",
    C1_name="flarnox",
    C1_detailed="",
    C1_values={"1": "bre", "0": "zin"},
    C2_name="drimbex",
    C2_detailed="",
    C2_values={"1": "clo", "0": "ven"},
    E_name="quorvex",
    E_detailed=".",
    E_values={"1": "sti", "0": "mol"},
    counterbalance_enabled=True,
)


This should re-create our rw17 dictionary. Let's verify this.

In [10]:
pprint.pprint(eldonia_domain_dict)

{'domain_name': 'Eldonia',
 'introduction': 'The mystical scholars of Eldonia seek to understand the '
                 'hidden forces that shape the arcane realm. They study key '
                 'magical attributes and their influence on one another, '
                 'unveiling the interconnected web of mystical energies.',
 'variables': {'C1': {'C1_detailed': 'Glyph resonance determines the strength '
                                     'with which ancient runes interact with '
                                     'magical conduits.',
                      'C1_name': 'glyph resonance',
                      'm_value': {'0': 'harmonized', '1': 'disrupted'},
                      'p_value': {'0': 'disrupted', '1': 'harmonized'}},
               'C2': {'C2_detailed': 'Ether tides are the fluctuating currents '
                                     'of raw magic that flow through the '
                                     'arcane realm, shifting unpredictably.',
                     

In [11]:
pprint.pprint(abstract_domain_dict)

{'domain_name': 'xqt',
 'introduction': '',
 'variables': {'C1': {'C1_detailed': '',
                      'C1_name': 'fwp',
                      'm_value': {'0': 'high', '1': 'low'},
                      'p_value': {'0': 'low', '1': 'high'}},
               'C2': {'C2_detailed': '',
                      'C2_name': 'blg',
                      'p_value': {'0': 'large', '1': 'small'}},
               'E': {'E_detailed': '',
                     'E_name': 'drk',
                     'm_value': {'0': 'large', '1': 'small'},
                     'p_value': {'0': 'small', '1': 'large'}}}}


## Step 2: Expand the Dictionaries into Prompt Components in Dataframe

In [12]:
# create a dataframe for each domain and then append them together

eldonia_df = expand_domain_to_dataframe(
    eldonia_domain_dict,
)
random_domain_df = expand_domain_to_dataframe(
    random_domain_dict,
)
abstract_domain_df = expand_domain_to_dataframe(
    abstract_domain_dict,
)
very_abstract_df = expand_domain_to_dataframe(
    very_abstract_domain_dict,
)


### Let's look at the dataframe:

In [13]:
print(
    f"Each domain dataframe now has {len(eldonia_df)} rows  \n and the following columns: \n {eldonia_df.columns}."
)
eldonia_df

Each domain dataframe now has 8 rows  
 and the following columns: 
 Index(['domain', 'C1', 'C1_values', 'C1_cntbl', 'C1_sense', 'C1_detailed',
       'C2', 'C2_values', 'C2_cntbl', 'C2_sense', 'C2_detailed', 'E',
       'E_values', 'E_cntbl', 'E_sense', 'E_detailed', 'cntbl_cond'],
      dtype='object').


,domain,C1,C1_values,C1_cntbl,C1_sense,C1_detailed,C2,C2_values,C2_cntbl,C2_sense,C2_detailed,E,E_values,E_cntbl,E_sense,E_detailed,cntbl_cond
0,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,ppp
1,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,ppm
2,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,pmp
3,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,pmm
4,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,mpp
5,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,mpm
6,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,mmp
7,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,mmm


In [14]:
eldonia_df

,domain,C1,C1_values,C1_cntbl,C1_sense,C1_detailed,C2,C2_values,C2_cntbl,C2_sense,C2_detailed,E,E_values,E_cntbl,E_sense,E_detailed,cntbl_cond
0,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,ppp
1,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,ppm
2,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,pmp
3,Eldonia,glyph resonance,1,p,harmonized,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,pmm
4,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,mpp
5,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,p,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,mpm
6,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,p,abundant,Mana crystallization refers to the process by ...,mmp
7,Eldonia,glyph resonance,1,m,disrupted,Glyph resonance determines the strength with w...,ether tides,1,m,stable,Ether tides are the fluctuating currents of ra...,mana crystallization,1,m,scarce,Mana crystallization refers to the process by ...,mmm



### How to derive the the off-sense of a variable:

Note that the function `expand_df_by_task_queries()` hands us back the necessary building blocks for verbalizing the entire prompt for every single of the 8 possible counterbalance conditions and every one of the 20 inference tasks. (Hence, the function `expand_df_by_task_queries()` will return a  dataframe with num_cntbl_conds * num_tasks, i.e. here: $8\cdot 20 = 160$ rows. Importanlty, the values for each variable node C1, C2, and E are only *1*, the variable-senses (high, low, small, etc.) for *0* are *not* contained in the dataframe. We have to look up the off-values (0) in the nested dictionary created by `create_domain_dict()`. The function `verbalize_inference_task()` does take care of this, when we want to verbalize an inference task that contains an off-value.

Below, you can verify for yourself that the function `verbalize_inference_task` correclty looks up off-variables  in the nested dictionary returnd  by `create_domain_dict()`. We first create the "building blocks"-df for the inference task verablization with `expand_df_by_task_queries`.

In [15]:
# create building blocks for inference task verbalization

random_domain_df_tasks_ex = expand_df_by_task_queries(
    random_domain_df, inference_tasks_rw17
).tail()
print("columns: ", random_domain_df_tasks_ex.columns)
random_domain_df_tasks_ex

columns:  Index(['domain', 'C1', 'C1_values', 'C1_cntbl', 'C1_sense', 'C1_detailed',
       'C2', 'C2_values', 'C2_cntbl', 'C2_sense', 'C2_detailed', 'E',
       'E_values', 'E_cntbl', 'E_sense', 'E_detailed', 'cntbl_cond', 'task',
       'query_node', 'observation', 'query'],
      dtype='object')


,domain,C1,C1_values,C1_cntbl,C1_sense,C1_detailed,C2,C2_values,C2_cntbl,C2_sense,...,E,E_values,E_cntbl,E_sense,E_detailed,cntbl_cond,task,query_node,observation,query
7,glooblarp,flimbor flux,1,m,unflarned,Flimbor flux determines the wavering intensity...,wibsnag oscillation,1,m,snarfulant,...,blorb containment,1,m,plimbled,Blorb containment is the ability to maintain a...,mmm,h,C2=1,"E=0, C1=0","p(C2=1|E=0, C1=0)"
7,glooblarp,flimbor flux,1,m,unflarned,Flimbor flux determines the wavering intensity...,wibsnag oscillation,1,m,snarfulant,...,blorb containment,1,m,plimbled,Blorb containment is the ability to maintain a...,mmm,i,E=1,"C1=0, C2=0","p(E=1|C1=0, C2=0)"
7,glooblarp,flimbor flux,1,m,unflarned,Flimbor flux determines the wavering intensity...,wibsnag oscillation,1,m,snarfulant,...,blorb containment,1,m,plimbled,Blorb containment is the ability to maintain a...,mmm,j,E=1,"C1=0, C2=1","p(E=1|C1=0, C2=1)"
7,glooblarp,flimbor flux,1,m,unflarned,Flimbor flux determines the wavering intensity...,wibsnag oscillation,1,m,snarfulant,...,blorb containment,1,m,plimbled,Blorb containment is the ability to maintain a...,mmm,j,E=1,"C2=0, C1=1","p(E=1|C2=0, C1=1)"
7,glooblarp,flimbor flux,1,m,unflarned,Flimbor flux determines the wavering intensity...,wibsnag oscillation,1,m,snarfulant,...,blorb containment,1,m,plimbled,Blorb containment is the ability to maintain a...,mmm,k,E=1,"C1=1, C2=1","p(E=1|C1=1, C2=1)"


Next, we verbalize a row from the above dataframe `econ_df_tasks_ex` that contains in the observation column "E=0, C1=0". You can verify for yourself that the function  `verbalize_inference_task` correctly looks up off-values in the dictionary `economy_domain_dict`.



***Customizing prompting Strategy:**
`verbalize_inference_task` also allows you to pass an argument `prompt_type` that allows you to customize the prompting strategy. The default is set to `prompt_type`=  `Please provide only a numeric response and no additional information.`. But you may want to try Chain-of-Thought (CoT) or tell the LLM to place its responses within certain characters like `< >`.



### A Note on the Counterbalance Conditions p and m
Those can be admittedly a bit confusing and you may think you don't need them. They are created automatically for you, so $2^3$ versions of combining p-m-p values. Simply subset for the ones you want to use. RW17 uses only 4 of the 8 possible combinations. 



## Creating the entire prompt for each condition and domain:

- Step 1:  for each domain dictionary, create the domain dataframe and then the verbalizations
- Step 2: append all complete dataframes
- Step 3: _optionally_ subset for counterbalance conditions of interest
- Step 4: _optionally_ merge with human data



In [16]:
abstract_domain_df.columns

Index(['domain', 'C1', 'C1_values', 'C1_cntbl', 'C1_sense', 'C1_detailed',
       'C2', 'C2_values', 'C2_cntbl', 'C2_sense', 'C2_detailed', 'E',
       'E_values', 'E_cntbl', 'E_sense', 'E_detailed', 'cntbl_cond'],
      dtype='object')

In [17]:
# create an empty dataframe to append the domain dataframes to
abstract_complete_df = generate_prompt_dataframe(
    domain_dict=abstract_domain_dict,
    inference_tasks=inference_tasks_rw17,
    graph_type="collider",
    graph_structures=graph_structures,
    counterbalance_enabled=True,
)

# list of remaining 2 omain dictionaries
domain_dicts = [random_domain_dict, eldonia_domain_dict, very_abstract_domain_dict]


# we'll start with the completed economy dataframe and append the other domain dataframes to it
new_over_complete_df = (
    abstract_complete_df.copy()  # over complete means, it has all 8 possible counterbalance conditions.
    # we'll subset later for the 4 counterbalance conditions that were used
)

for dict in domain_dicts:
    # for dict in rw_17_domain_components.values():
    df = generate_prompt_dataframe(
        domain_dict=dict,
        inference_tasks=inference_tasks_rw17,
        graph_type="collider",
        graph_structures=graph_structures,
        counterbalance_enabled=True,
    )
    # append the new dataframe to the complete dataframe
    new_over_complete_df = append_dfs(new_over_complete_df, df)

### Subsetting for the Counterbalance conditions used in RW17
- and adding unique id.

In [18]:
# add a unique id to each row
new_over_complete_df["id"] = range(1, len(new_over_complete_df) + 1)

# subset for the counterbalance conditions used in the RW17 study
select_contbl_cond_xs = ["ppp", "pmm", "mmp", "mpm"]

In [19]:
new_complete_df = new_over_complete_df[
    new_over_complete_df["cntbl_cond"].isin(select_contbl_cond_xs)
]

# save new_complete_df to a csv file#
prompt_category = new_complete_df["prompt_category"].unique()[0]
graph_type = new_complete_df["graph"].unique()[0]
new_complete_df.to_csv(
    f"../datasets/abstract_collider_prompts/abstract_{prompt_category}_LLM_prompting_{graph_type}_abstract.csv",
    index=False,
)


In [20]:
print(len(new_complete_df))
print(new_complete_df.columns)
print(new_complete_df["task"].unique())
print(new_complete_df["domain"].unique())


320
Index(['domain', 'C1', 'C1_values', 'C1_cntbl', 'C1_sense', 'C1_detailed',
       'C2', 'C2_values', 'C2_cntbl', 'C2_sense', 'C2_detailed', 'E',
       'E_values', 'E_cntbl', 'E_sense', 'E_detailed', 'cntbl_cond', 'task',
       'query_node', 'observation', 'query', 'graph', 'prompt',
       'prompt_category', 'id'],
      dtype='object')
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k']
['xqt' 'glooblarp' 'Eldonia' 'zorpentix']


## Saving Dataframes to csv

### To prompt the the LLMs, we only need the id and prompt.
- to not confuse the LLMs, we should group them by counterbalance condition when interacting with them through the website.
- however, the default when interacting with them throught their respective APIs is that they're stateless, meaning they don't retain anything from the previous context.
    - this is why we don't have to worry about having the different counterbalance conditions contradict each other and hence confuse the LLMs


In [21]:
# save for prompting LLMs
LLM_prompting_df = new_complete_df[
    ["id", "prompt", "prompt_category", "graph", "domain", "cntbl_cond", "task"]
]
prompt_category = LLM_prompting_df["prompt_category"].unique()[0]
graph_type = LLM_prompting_df["graph"].unique()[0]
print(prompt_category, graph_type)
LLM_prompting_df.to_csv(
    f"../datasets/abstract_collider_prompts/prompts_for_LLM_api/{prompt_category}_LLM_prompting_{graph_type}_abstract.csv",
    index=False,
)

print("Prompts for LLMs saved successfully!")
print("prompt data file has the following columns:")
print(LLM_prompting_df.columns)

single_numeric_response collider
Prompts for LLMs saved successfully!
prompt data file has the following columns:
Index(['id', 'prompt', 'prompt_category', 'graph', 'domain', 'cntbl_cond',
       'task'],
      dtype='object')
